In [ ]:
# 🚀 PRODUCTION SERVING ENDPOINT - NEW WORKFLOW (CONFIG-DRIVEN)

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
    EndpointCoreConfigInput,
    ServedEntityInput
)
import mlflow
from mlflow.tracking import MlflowClient
import time
import sys
import yaml
import traceback
import requests
from typing import Optional, Dict, Tuple
from datetime import datetime

print("=" * 80)
print("🚀 PRODUCTION SERVING ENDPOINT (NEW WORKFLOW)")
print("=" * 80)
 
# ✅ LOAD PIPELINE CONFIGURATION

print("\n📋 Step 1: Loading configuration from pipeline_config.yml...")

try:
    import os

    config_path = "/Workspace/Repos/vipultak7171@gmail.com/ml-credit-risk/dev_env/pipeline_config.yml"

    if not os.path.exists(config_path):
        config_path = "/Workspace/ml-credit-risk/dev_env/pipeline_config.yml"

    with open(config_path, "r") as f:
        pipeline_cfg = yaml.safe_load(f)

    print(f"✅ Configuration loaded successfully from: {config_path}")

except FileNotFoundError:
    print("❌ ERROR: pipeline_config.yml not found!")
    print("💡 Please ensure pipeline_config.yml is in the notebook directory")
    sys.exit(1)
except Exception as e:
    print(f"❌ ERROR loading configuration: {e}")
    traceback.print_exc()
    sys.exit(1)
 
# ✅ CONFIGURATION CLASS

class Config:
    """Configuration manager - reads from pipeline_config.yml"""
    
    def __init__(self):
        MODEL_TYPE = pipeline_cfg["model"]["type"]
        UC_CATALOG = pipeline_cfg["model"]["catalog"]
        UC_SCHEMA = pipeline_cfg["model"]["schema"]
        BASE_NAME = pipeline_cfg["model"]["base_name"]
        
        self.MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.{BASE_NAME}_{MODEL_TYPE}"
        self.MODEL_TYPE = MODEL_TYPE
        
        self.ENDPOINT_NAME = f"{BASE_NAME.replace('_', '-')}-{MODEL_TYPE}-prod"
        
        self.PRODUCTION_ALIAS = pipeline_cfg["aliases"]["production"]
        self.STAGING_ALIAS = pipeline_cfg["aliases"]["staging"]
        
        serving_cfg = pipeline_cfg.get("serving", {})
        self.WORKLOAD_SIZE = serving_cfg.get("workload_size", "Small")
        self.SCALE_TO_ZERO = serving_cfg.get("scale_to_zero_enabled", True)
        self.MIN_PROVISIONED_THROUGHPUT = serving_cfg.get("min_provisioned_throughput", 0)
        self.MAX_PROVISIONED_THROUGHPUT = serving_cfg.get("max_provisioned_throughput", 0)
        
        self.WORKLOAD_TYPE = serving_cfg.get("workload_type", "CPU")
        self.ENVIRONMENT_VARS = serving_cfg.get("environment_vars", {})
        
        self.DEPLOYMENT_TIMEOUT = serving_cfg.get("deployment_timeout", 1800)
        self.STATUS_CHECK_INTERVAL = serving_cfg.get("status_check_interval", 15)
        
        self.SLACK_ENABLED = pipeline_cfg["notifications"]["enabled"]
        self.SLACK_WEBHOOK_URL = self._get_slack_webhook()
        
        print(f"\n📊 Configuration Summary:")
        print(f"   Model Type: {self.MODEL_TYPE.upper()}")
        print(f"   Model Name: {self.MODEL_NAME}")
        print(f"   Endpoint Name: {self.ENDPOINT_NAME}")
        print(f"   Production Alias: @{self.PRODUCTION_ALIAS}")
        print(f"   Workload Size: {self.WORKLOAD_SIZE}")
        print(f"   Workload Type: {self.WORKLOAD_TYPE}")
        print(f"   Scale to Zero: {self.SCALE_TO_ZERO}")
        print(f"   Deployment Timeout: {self.DEPLOYMENT_TIMEOUT}s")
        print(f"   Slack: {'ENABLED' if self.SLACK_WEBHOOK_URL else 'DISABLED'}")
    
    def _get_slack_webhook(self) -> Optional[str]:
        if not self.SLACK_ENABLED:
            print("⚠️ Slack disabled in config.yml")
            return None
        
        print("\n🔍 Looking for Slack Secret...")
        scopes = ["shared-scope", "dev-scope", "prod-scope", "ml-scope"]

        for scope in scopes:
            try:
                webhook = dbutils.secrets.get(scope, "SLACK_WEBHOOK_URL")
                if webhook:
                    print(f"   ✅ Slack webhook found in scope: '{scope}'")
                    return webhook
            except Exception as err:
                print(f"   ⚠️ Scope '{scope}' lookup failed → {err}")
                continue
        
        print("❌ No Slack webhook found — notification disabled.")
        return None

config = Config()

print("=" * 80)

# 📢 SLACK NOTIFICATION HELPER

class SlackNotifier:
    """Slack notification handler"""
    
    def __init__(self, webhook_url: Optional[str]):
        self.webhook_url = webhook_url
        self.enabled = webhook_url is not None and webhook_url.strip() != ""
        
        if not self.enabled:
            print("⚠️ Slack notifications OFF — No webhook available.")

    def send(self, message: str, level: str = "info", extra_fields: Optional[Dict] = None) -> bool:
        """Send Slack notification"""
        if not self.enabled:
            print(f"📢 [Slack Skipped] {message}")
            return False
        
        emoji_map = {
            "info": "ℹ️",
            "success": "✅",
            "warning": "⚠️",
            "error": "❌",
            "rocket": "🚀",
            "gear": "⚙️"
        }
        
        formatted_message = f"{emoji_map.get(level, 'ℹ️')} *{message}*"
        
        if extra_fields:
            formatted_message += "\n"
            for key, value in extra_fields.items():
                formatted_message += f"\n• *{key}:* {value}"
        
        payload = {
            "text": formatted_message,
            "username": "ML Pipeline Bot",
            "icon_emoji": ":robot_face:"
        }
        
        try:
            response = requests.post(self.webhook_url, json=payload)
            print(f"📨 Slack → Status: {response.status_code}")
            return response.status_code == 200
        
        except Exception as e:
            print(f"❌ Slack notification failed: {e}")
            return False

    def send_deployment_started(self, endpoint_name: str, model_version: int):
        return self.send(
            "🚀 Deployment Started",
            "gear",
            {"Endpoint": endpoint_name, "Model Version": model_version}
        )

    def send_deployment_success(self, endpoint_name: str, model_version: int, time_taken: float):
        return self.send(
            "🎉 Deployment Completed",
            "rocket",
            {"Endpoint": endpoint_name, "Version": model_version, "Time": f"{time_taken:.2f}s"}
        )

    def send_deployment_failed(self, endpoint_name: str, reason: str):
        return self.send(
            "❌ Deployment Failed",
            "error",
            {"Endpoint": endpoint_name, "Reason": reason}
        )


# INIT notifier
slack = SlackNotifier(config.SLACK_WEBHOOK_URL)

